# LangChain Prompts: Comprehensive Prompt Engineering

## Introduction

**Prompt engineering** is the art and science of crafting effective inputs for language models. LangChain provides powerful tools to create reusable, composable prompt templates.

### What Are Prompt Templates?

Prompt templates allow you to:
- **Separate logic from content**: Keep prompts reusable and maintainable
- **Use variables**: Dynamic prompts with placeholders
- **Compose prompts**: Build complex prompts from smaller pieces
- **Format consistently**: System, Human, AI messages properly structured
- **Parse outputs**: Extract structured data from LLM responses

### Why Prompt Templates?

| ❌ Without Templates | ✅ With Templates |
|---------------------|------------------|
| f"Translate {text} to {lang}" | template.format(text=text, lang=lang) |
| Hardcoded strings | Reusable components |
| No validation | Type-checked inputs |
| Manual message formatting | Automatic ChatMessage creation |

---

## Installation & Setup

In [ ]:
import os
from getpass import getpass

# Set API key
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API Key: ")

print("API key configured!")

---

## Example 1: Basic ChatPromptTemplate

The most common prompt template type for chat models:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Create a simple template
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")

# Inspect what it creates
formatted = prompt.format(topic="programming")
print("Formatted prompt:")
print(formatted)

# Use in a chain
chain = prompt | ChatOpenAI(model="gpt-4") | StrOutputParser()
result = chain.invoke({"topic": "Python"})
print(f"\nResult:\n{result}")

---

## Example 2: System + Human Messages

Create structured prompts with system instructions and user input:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Create a template with multiple message types
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful {role} assistant with expertise in {domain}."),
    ("human", "{user_input}")
])

# See the template structure
print("Template structure:")
print(prompt)

# Format with variables
messages = prompt.format_messages(
    role="Python programming",
    domain="software design patterns",
    user_input="Explain the factory pattern"
)

print("\nFormatted messages:")
for msg in messages:
    print(f"{msg.__class__.__name__}: {msg.content}")

---

## Example 3: Multiple Message Types

Include examples of previous conversations (few-shot learning):

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Template with examples
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment classifier. Respond with only: positive, negative, or neutral"),
    ("human", "I love this product!"),
    ("ai", "positive"),
    ("human", "This is terrible."),
    ("ai", "negative"),
    ("human", "It's okay."),
    ("ai", "neutral"),
    ("human", "{text}")
])

# Create chain
chain = prompt | ChatOpenAI(model="gpt-4", temperature=0) | StrOutputParser()

# Test with new examples
test_texts = [
    "This is amazing!",
    "I hate waiting in lines.",
    "The weather is nice today."
]

for text in test_texts:
    sentiment = chain.invoke({"text": text})
    print(f"{text} -> {sentiment}")

---

## Example 4: MessagesPlaceholder

For dynamic conversation history:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI

# Template with placeholder for chat history
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer based on conversation context."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{user_input}")
])

# Create conversation history
history = [
    HumanMessage(content="My name is Alice"),
    AIMessage(content="Hello Alice! How can I help you today?"),
    HumanMessage(content="I like Python programming"),
    AIMessage(content="That's great! Python is a versatile language.")
]

# Create chain
chain = prompt | ChatOpenAI(model="gpt-4")

# Ask question that requires context
response = chain.invoke({
    "chat_history": history,
    "user_input": "What's my name and what do I like?"
})

print(response.content)

---

## Example 5: Few-Shot Prompting

Provide examples to guide the model's behavior:

In [ ]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Define examples
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "hot", "output": "cold"},
    {"input": "fast", "output": "slow"}
]

# Create example template
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

# Create few-shot template
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# Create final prompt
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an antonym generator. Respond with only the antonym."),
    few_shot_prompt,
    ("human", "{input}")
])

# Create chain
chain = final_prompt | ChatOpenAI(model="gpt-4", temperature=0) | StrOutputParser()

# Test it
result = chain.invoke({"input": "big"})
print(f"big -> {result}")

---

## Example 6: Output Parsers - StrOutputParser

Extract string content from responses:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Tell me about {topic}")
model = ChatOpenAI(model="gpt-4")

# Without output parser - returns AIMessage object
chain_without_parser = prompt | model
result = chain_without_parser.invoke({"topic": "Python"})
print("Without parser:")
print(f"Type: {type(result)}")
print(f"Content: {result.content[:100]}...\n")

# With output parser - returns string
chain_with_parser = prompt | model | StrOutputParser()
result = chain_with_parser.invoke({"topic": "Python"})
print("With parser:")
print(f"Type: {type(result)}")
print(f"Content: {result[:100]}...")

---

## Example 7: JsonOutputParser

Parse JSON from LLM responses:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

# Create parser
parser = JsonOutputParser()

# Create prompt with format instructions
prompt = ChatPromptTemplate.from_template(
    "Extract information about {person}.\n{format_instructions}\n"
    "Return JSON with keys: name, occupation, nationality"
)

# Add format instructions to prompt
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# Create chain
chain = prompt | ChatOpenAI(model="gpt-4") | parser

# Invoke
result = chain.invoke({"person": "Marie Curie"})
print("Result:")
print(result)
print(f"\nType: {type(result)}")
print(f"Name: {result['name']}")

---

## Example 8: PydanticOutputParser

Parse into validated Pydantic models:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from typing import List

# Define output schema
class Person(BaseModel):
    name: str = Field(description="Person's full name")
    age: int = Field(description="Person's age in years")
    occupation: str = Field(description="Person's job or profession")
    skills: List[str] = Field(description="List of skills")
    
    @validator('age')
    def age_must_be_positive(cls, v):
        if v <= 0:
            raise ValueError('Age must be positive')
        return v

# Create parser
parser = PydanticOutputParser(pydantic_object=Person)

# Create prompt
prompt = ChatPromptTemplate.from_template(
    "Generate information about a fictional {profession}.\n{format_instructions}"
)

# Add format instructions
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# Create chain
chain = prompt | ChatOpenAI(model="gpt-4") | parser

# Invoke
result = chain.invoke({"profession": "software engineer"})
print("Result:")
print(f"Type: {type(result)}")
print(f"Name: {result.name}")
print(f"Age: {result.age}")
print(f"Occupation: {result.occupation}")
print(f"Skills: {', '.join(result.skills)}")

---

## Example 9: Partial Prompts

Pre-fill some variables, provide others later:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime

# Create template with multiple variables
prompt = ChatPromptTemplate.from_template(
    "You are a {role} assistant. Today is {date}.\n\nUser question: {question}"
)

# Partially fill the template
partial_prompt = prompt.partial(
    role="Python programming",
    date=datetime.now().strftime("%Y-%m-%d")
)

# Create chain (role and date are already filled)
chain = partial_prompt | ChatOpenAI(model="gpt-4") | StrOutputParser()

# Only need to provide 'question' now
result = chain.invoke({"question": "What are decorators?"})
print(result)

---

## Example 10: Partial with Functions

Use functions to generate dynamic values:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime

# Function to get current time
def get_current_time():
    return datetime.now().strftime("%H:%M:%S")

# Create template
prompt = ChatPromptTemplate.from_template(
    "Current time: {time}\n\nUser: {input}"
)

# Partially fill with function
partial_prompt = prompt.partial(time=get_current_time)

# Create chain
chain = partial_prompt | ChatOpenAI(model="gpt-4") | StrOutputParser()

# The time will be evaluated at invocation
print("First call:")
result1 = chain.invoke({"input": "What time is it?"})
print(result1)

import time
time.sleep(2)

print("\nSecond call (2 seconds later):")
result2 = chain.invoke({"input": "What time is it now?"})
print(result2)

---

## Example 11: Prompt Composition

Build complex prompts from smaller pieces:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Create reusable prompt components
system_template = "You are an expert in {domain}. Answer questions clearly and concisely."
context_template = "Context: {context}"
question_template = "Question: {question}"

# Compose them
prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", context_template + "\n\n" + question_template)
])

# Create chain
chain = prompt | ChatOpenAI(model="gpt-4") | StrOutputParser()

# Use with different domains and contexts
result = chain.invoke({
    "domain": "Python programming",
    "context": "Python has built-in support for async/await syntax.",
    "question": "What are the benefits of using async/await?"
})

print(result)

---

## Example 12: Pipeline Prompts

Use one prompt's output as another's input:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# First prompt: Generate a topic
topic_prompt = ChatPromptTemplate.from_template(
    "Generate a creative topic related to {category}"
)

# Second prompt: Write about that topic
writing_prompt = ChatPromptTemplate.from_template(
    "Write a short paragraph about: {topic}"
)

model = ChatOpenAI(model="gpt-4")
parser = StrOutputParser()

# Create two-stage chain
topic_chain = topic_prompt | model | parser
writing_chain = writing_prompt | model | parser

# Execute stages
topic = topic_chain.invoke({"category": "artificial intelligence"})
print(f"Generated topic: {topic}\n")

paragraph = writing_chain.invoke({"topic": topic})
print(f"Paragraph:\n{paragraph}")

---

## Example 13: Custom Output Parser

Create your own parser for specific formats:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import BaseOutputParser
from typing import List

# Custom parser for comma-separated lists
class CommaSeparatedListParser(BaseOutputParser[List[str]]):
    """Parse comma-separated list."""
    
    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        return [item.strip() for item in text.split(",")]
    
    def get_format_instructions(self) -> str:
        return "Your response should be a comma-separated list, like: item1, item2, item3"

# Create parser instance
parser = CommaSeparatedListParser()

# Create prompt
prompt = ChatPromptTemplate.from_template(
    "List 5 {category}.\n{format_instructions}"
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# Create chain
chain = prompt | ChatOpenAI(model="gpt-4", temperature=0.7) | parser

# Invoke
result = chain.invoke({"category": "programming languages"})
print(f"Type: {type(result)}")
print(f"Items: {result}")
print(f"Count: {len(result)}")

---

## Example 14: Structured Output with JSON Mode

Force OpenAI models to return valid JSON:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
import json

# Create prompt that explicitly asks for JSON
prompt = ChatPromptTemplate.from_template(
    "Extract structured information about {topic}.\n"
    "Return JSON with keys: name, description, examples (list), difficulty (beginner/intermediate/advanced)"
)

# Enable JSON mode (OpenAI specific)
model = ChatOpenAI(
    model="gpt-4",
    model_kwargs={"response_format": {"type": "json_object"}}
)

parser = JsonOutputParser()

# Create chain
chain = prompt | model | parser

# Invoke
result = chain.invoke({"topic": "Python decorators"})
print("Parsed result:")
print(json.dumps(result, indent=2))

---

## Best Practices

### ✅ Do

1. **Use ChatPromptTemplate** for chat models (not PromptTemplate)
2. **Provide clear system messages** to set behavior
3. **Use few-shot examples** for complex tasks
4. **Validate outputs** with Pydantic parsers
5. **Use partial prompts** for dynamic values
6. **Compose prompts** from reusable components
7. **Include format instructions** from parsers

### ❌ Don't

1. **Don't use f-strings** directly (use templates for validation)
2. **Don't forget to handle parsing errors**
3. **Don't over-complicate prompts** (keep them focused)
4. **Don't hardcode examples** (use variables)
5. **Don't ignore format instructions** from parsers

---

## Common Pitfalls

### ❌ Mistake 1: Wrong Parser for Chat Models

```python
# Wrong - PromptTemplate for chat model
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template("Hello {name}")
```

**Solution**: Use `ChatPromptTemplate` for chat models.

### ❌ Mistake 2: Missing Variables

```python
# Template has {topic} but we don't provide it
prompt = ChatPromptTemplate.from_template("Tell me about {topic}")
prompt.invoke({})  # Error!
```

**Solution**: Always provide all required variables.

### ❌ Mistake 3: Not Using Format Instructions

```python
# Parser expects specific format but prompt doesn't specify it
parser = JsonOutputParser()
prompt = ChatPromptTemplate.from_template("Tell me about {topic}")
```

**Solution**: Use `parser.get_format_instructions()` in your prompt.

---

## Practice Exercises

In [ ]:
# Exercise 1: Create a translation chain with language detection
# Input: text to translate, target language
# Output: Pydantic model with source_language, target_language, translated_text

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# Your code here:
class Translation(BaseModel):
    source_language: str = Field(description="Detected source language")
    target_language: str = Field(description="Target language")
    translated_text: str = Field(description="Translated text")

parser = PydanticOutputParser(pydantic_object=Translation)

prompt = ChatPromptTemplate.from_template(
    "Translate the following text to {target_language}.\n"
    "Text: {text}\n\n"
    "{format_instructions}"
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | ChatOpenAI(model="gpt-4") | parser

result = chain.invoke({
    "text": "Hello, how are you?",
    "target_language": "Spanish"
})

print(f"Source: {result.source_language}")
print(f"Target: {result.target_language}")
print(f"Translation: {result.translated_text}")

In [ ]:
# Exercise 2: Create a code review chain
# Input: code snippet
# Output: Structured feedback with issues (list), suggestions (list), rating (1-10)

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

# Your code here:
class CodeReview(BaseModel):
    issues: List[str] = Field(description="List of issues found")
    suggestions: List[str] = Field(description="List of improvement suggestions")
    rating: int = Field(description="Code quality rating from 1-10", ge=1, le=10)

parser = PydanticOutputParser(pydantic_object=CodeReview)

prompt = ChatPromptTemplate.from_template(
    "Review this Python code and provide detailed feedback:\n\n"
    "```python\n{code}\n```\n\n"
    "{format_instructions}"
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | ChatOpenAI(model="gpt-4") | parser

code_sample = """
def calculate(a, b):
    return a + b
"""

result = chain.invoke({"code": code_sample})
print(f"Rating: {result.rating}/10")
print(f"Issues: {result.issues}")
print(f"Suggestions: {result.suggestions}")

In [ ]:
# Exercise 3: Create a multi-stage research chain
# Stage 1: Generate 3 research questions about a topic
# Stage 2: Answer each question
# Stage 3: Synthesize into a summary

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

# Your code here:
model = ChatOpenAI(model="gpt-4")

# Stage 1: Generate questions
questions_prompt = ChatPromptTemplate.from_template(
    "Generate 3 specific research questions about {topic}.\n"
    "Return as JSON array with key 'questions'."
)
questions_chain = questions_prompt | model | JsonOutputParser()

# Stage 2: Answer questions
answer_prompt = ChatPromptTemplate.from_template(
    "Answer this question concisely: {question}"
)
answer_chain = answer_prompt | model | StrOutputParser()

# Stage 3: Synthesize
synthesis_prompt = ChatPromptTemplate.from_template(
    "Synthesize these Q&A pairs into a cohesive summary about {topic}:\n\n{qa_pairs}"
)
synthesis_chain = synthesis_prompt | model | StrOutputParser()

# Execute pipeline
topic = "Python async/await"

# Get questions
questions_result = questions_chain.invoke({"topic": topic})
questions = questions_result["questions"]
print("Questions:")
for i, q in enumerate(questions, 1):
    print(f"{i}. {q}")

# Answer each
qa_pairs = []
for q in questions:
    answer = answer_chain.invoke({"question": q})
    qa_pairs.append(f"Q: {q}\nA: {answer}")

# Synthesize
summary = synthesis_chain.invoke({
    "topic": topic,
    "qa_pairs": "\n\n".join(qa_pairs)
})

print(f"\nSummary:\n{summary}")

---

## Key Takeaways

### ✅ What We Learned

1. **ChatPromptTemplate**: Create reusable, composable prompts
2. **Message Types**: System, Human, AI messages for context
3. **MessagesPlaceholder**: Dynamic conversation history
4. **Few-Shot Prompting**: Guide behavior with examples
5. **Output Parsers**: Extract structured data (String, JSON, Pydantic)
6. **Partial Prompts**: Pre-fill variables, add others later
7. **Prompt Composition**: Build complex prompts from components
8. **Format Instructions**: Tell LLM how to format output

### 📚 Next Steps

- **langchain_lcel.ipynb**: Advanced chain composition
- **langchain_rag.ipynb**: Use prompts in RAG pipelines
- **langchain_agents.ipynb**: Dynamic prompts with agents

---

## Resources

- [Prompt Engineering Guide](https://www.promptingguide.ai/)
- [LangChain Prompts Documentation](https://python.langchain.com/docs/modules/model_io/prompts/)
- [Output Parsers Documentation](https://python.langchain.com/docs/modules/model_io/output_parsers/)
- [Few-Shot Prompting](https://www.promptingguide.ai/techniques/fewshot)

---

**Next Notebook**: `langchain_lcel.ipynb` - Master LCEL for advanced chain composition